In [43]:
import pandas as pd
import numpy as np

Обязательная часть
Будем осуществлять работу с непростым набором данных о состоянии здоровья лошадей, испытывающих кишечные колики.

Задание 1. Базовое изучение
Изучить представленный набор данных на основе описания его столбцов и выбрать 8 столбцов для дальнейшего изучения (среди них должны быть как числовые, так и категориальные). Провести расчет базовых метрик для них, кратко описать результаты.

In [126]:
horse = pd.read_csv('horse_data.csv', na_values='?')
#именуем столбцы в соответствии с файлом name
horse.columns = (['surgery',
                       'Age',
                        'Hospital Number',
                       'rectal temperature',
                        'pulse',
                       'respiratory rate',
                        'temperature of extremities',
                       'peripheral pulse',
                        'mucous membranes',
                       'capillary refill time',
                        'pain',
                       'peristalsis',
                        'abdominal distension',
                       'nasogastric tube',
                        'nasogastric reflux',
                       'nasogastric reflux PH',
                        'rectal examination',
                       'abdomen',
                        'packed cell volume',
                       'total protein',
                        'abdominocentesis appearance',
                       'abdomcentesis total protein',
                        'outcome',
                       'surgical lesion?',
                        'type of lesion',
                       '25',
                        '26',
                       'cp_data'])
# выбираем 8 столбцов
horse8 = horse[['Hospital Number','Age','type of lesion','surgery','rectal temperature','pulse','total protein','outcome']]
mode_dict = {}
# т.к. в .describe() нет моды, то считаем моду для каждого столбца 
for col in horse8.columns:
    mode = horse8[col].mode()[0]
#моды складываем в словарь
    mode_dict.setdefault(col,mode)
    print(f'Мода столбца {col} = {mode}')
print(mode_dict)
# объединяем датафрейм из основных значений  из .describe() и датафрейм из словаря с модами
res = pd.concat([horse8.describe(), pd.DataFrame([mode_dict])]).round(2)
# преименовываем индексы, чтобы указать индекс 'mode'
res.index = ['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max', 'mode']
horse8.info()
res



Мода столбца Hospital Number = 527544
Мода столбца Age = 1
Мода столбца type of lesion = 0
Мода столбца surgery = 1.0
Мода столбца rectal temperature = 38.0
Мода столбца pulse = 48.0
Мода столбца total protein = 6.5
Мода столбца outcome = 1.0
{'Hospital Number': 527544, 'Age': 1, 'type of lesion': 0, 'surgery': 1.0, 'rectal temperature': 38.0, 'pulse': 48.0, 'total protein': 6.5, 'outcome': 1.0}
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Hospital Number     299 non-null    int64  
 1   Age                 299 non-null    int64  
 2   type of lesion      299 non-null    int64  
 3   surgery             298 non-null    float64
 4   rectal temperature  239 non-null    float64
 5   pulse               275 non-null    float64
 6   total protein       266 non-null    float64
 7   outcome             298 non-null    float64
dtypes:

,Hospital Number,Age,type of lesion,surgery,rectal temperature,pulse,total protein,outcome
count,299.00,299.00,299.00,298.00,239.00,275.00,266.00,298.00
mean,1087747.66,1.64,3632.32,1.40,38.17,71.93,24.52,1.55
std,1532026.14,2.18,5390.36,0.49,0.73,28.68,27.51,0.74
min,518476.00,1.00,0.00,1.00,35.40,30.00,3.30,1.00
25%,528904.00,1.00,2111.50,1.00,37.80,48.00,6.50,1.00
50%,530310.00,1.00,2322.00,1.00,38.20,64.00,7.50,1.00
75%,534736.00,1.00,3209.00,2.00,38.50,88.00,57.00,2.00
max,5305629.00,9.00,41110.00,2.00,40.80,184.00,89.00,3.00
mode,527544.00,1.00,0.00,1.00,38.00,48.00,6.50,1.00


Задание 2. Работа с выбросами
В выбранных числовых столбцах найти выбросы, выдвинуть гипотезы об их причинах и проинтерпретировать результаты. Принять и обосновать решение о дальнейшей работе с ними.

In [127]:
q1 = horse8.surgery.quantile(0.25)
q3 = horse8.surgery.quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
remove_outliers = horse8[horse8.surgery.between(lower_bound, upper_bound, inclusive=True)].sort_values('pulse')
pd.concat([horse8, remove_outliers]).drop_duplicates(keep=False)

,Hospital Number,Age,type of lesion,surgery,rectal temperature,pulse,total protein,outcome
131,534572,1,3111,NaN,38.0,48.0,73.0,NaN


In [128]:
q1 = horse8.Age.quantile(0.25)
q3 = horse8.Age.quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
remove_outliers = horse8[horse8.Age.between(lower_bound, upper_bound, inclusive=True)].sort_values('pulse')
pd.concat([horse8, remove_outliers]).drop_duplicates(keep=False)

,Hospital Number,Age,type of lesion,surgery,rectal temperature,pulse,total protein,outcome
2,5290409,9,2208,1.0,39.1,164.0,7.2,2.0
8,5299629,9,0,2.0,38.3,90.0,6.2,1.0
12,5291329,9,0,2.0,38.0,92.0,6.1,2.0
15,5301219,9,1400,1.0,NaN,128.0,7.8,2.0
22,534998,9,3111,1.0,38.3,130.0,70.0,1.0
38,5277409,9,2113,1.0,39.2,146.0,NaN,2.0
40,5288249,9,9400,2.0,39.0,150.0,8.5,1.0
54,5282839,9,7111,1.0,38.6,160.0,NaN,2.0
73,5292929,9,11124,1.0,NaN,NaN,4.9,2.0
74,534092,9,1400,1.0,39.7,100.0,57.0,3.0


In [129]:
q1 = horse8['rectal temperature'].quantile(0.25)
q3 = horse8['rectal temperature'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
remove_outliers = horse8[horse8['rectal temperature'].between(lower_bound, upper_bound, inclusive=True)].sort_values('pulse')
pd.concat([horse8, remove_outliers]).drop_duplicates(keep=False)

,Hospital Number,Age,type of lesion,surgery,rectal temperature,pulse,total protein,outcome
4,528355,1,0,2.0,NaN,NaN,NaN,1.0
6,529607,1,2208,1.0,NaN,60.0,8.3,2.0
7,530051,1,3205,2.0,NaN,80.0,6.2,3.0
15,5301219,9,1400,1.0,NaN,128.0,7.8,2.0
19,530157,1,2111,1.0,39.9,72.0,6.1,1.0
...,...,...,...,...,...,...,...,...
281,527544,1,1400,1.0,NaN,70.0,7.5,2.0
287,529428,1,2124,1.0,NaN,NaN,NaN,1.0
292,534004,1,2209,1.0,NaN,78.0,62.0,3.0
294,533886,1,3205,1.0,NaN,120.0,65.0,3.0


In [130]:
q1 = horse8['pulse'].quantile(0.25)
q3 = horse8['pulse'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
remove_outliers = horse8[horse8['pulse'].between(lower_bound, upper_bound, inclusive=True)].sort_values('pulse')
pd.concat([horse8, remove_outliers]).drop_duplicates(keep=False)

,Hospital Number,Age,type of lesion,surgery,rectal temperature,pulse,total protein,outcome
2,5290409,9,2208,1.0,39.1,164.0,7.2,2.0
4,528355,1,0,2.0,NaN,NaN,NaN,1.0
27,5279442,1,4300,1.0,NaN,NaN,NaN,2.0
40,5288249,9,9400,2.0,39.0,150.0,8.5,1.0
51,529483,1,3111,2.0,NaN,NaN,7.7,1.0
54,5282839,9,7111,1.0,38.6,160.0,NaN,2.0
55,528872,1,3112,1.0,NaN,NaN,6.7,1.0
57,528298,1,3205,1.0,NaN,NaN,5.9,2.0
73,5292929,9,11124,1.0,NaN,NaN,4.9,2.0
82,5279822,1,3205,1.0,38.0,NaN,7.8,2.0


In [131]:
q1 = horse8['total protein'].quantile(0.25)
q3 = horse8['total protein'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
remove_outliers = horse8[horse8['total protein'].between(lower_bound, upper_bound, inclusive=True)].sort_values('pulse')
pd.concat([horse8, remove_outliers]).drop_duplicates(keep=False)

,Hospital Number,Age,type of lesion,surgery,rectal temperature,pulse,total protein,outcome
4,528355,1,0,2.0,NaN,NaN,NaN,1.0
16,526639,1,0,2.0,37.5,48.0,NaN,1.0
24,529518,1,0,2.0,37.8,60.0,NaN,1.0
27,5279442,1,4300,1.0,NaN,NaN,NaN,2.0
38,5277409,9,2113,1.0,39.2,146.0,NaN,2.0
50,527706,1,7209,1.0,37.4,84.0,NaN,2.0
54,5282839,9,7111,1.0,38.6,160.0,NaN,2.0
58,528904,1,4205,2.0,NaN,96.0,NaN,2.0
71,5299253,1,2113,1.0,37.7,56.0,NaN,2.0
80,534756,1,3111,1.0,37.3,40.0,NaN,1.0


Выбросы все оставляем , т.к. это валидные значения или NaN, кроме 'Age', в котором значение 9 невалидно и не соответствует нотификации в файле name. Можно '9' заменить на NaN, а можно оставить и так, т.к. среднее для этих значений(диcкретные 1 или 2)
не имеет смысла

In [51]:
horse8[horse8['Age']==9].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24 entries, 2 to 286
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Hospital Number     24 non-null     int64  
 1   Age                 24 non-null     int64  
 2   type of lesion      24 non-null     int64  
 3   surgery             24 non-null     float64
 4   rectal temperature  20 non-null     float64
 5   pulse               22 non-null     float64
 6   total protein       22 non-null     float64
 7   outcome             24 non-null     float64
dtypes: float64(5), int64(3)
memory usage: 1.7 KB


In [52]:
horse9 = horse8.copy()
horse9['Age'].replace(9, np.nan, inplace=True)
horse9[horse9['Age'].isna()==True].info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 24 entries, 2 to 286
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Hospital Number     24 non-null     int64  
 1   Age                 0 non-null      float64
 2   type of lesion      24 non-null     int64  
 3   surgery             24 non-null     float64
 4   rectal temperature  20 non-null     float64
 5   pulse               22 non-null     float64
 6   total protein       22 non-null     float64
 7   outcome             24 non-null     float64
dtypes: float64(6), int64(2)
memory usage: 1.7 KB


Задание 3. Работа с пропусками
Рассчитать количество пропусковдля всех выбранных столбцов. Принять и обосновать решение о методе работы с пропусками по каждому столбцу, сформировать датафрейм, в котором пропуски будут отсутствовать.

In [57]:
horse8[horse8['Hospital Number'].isna()==True]
#нет пропусков

,Hospital Number,Age,type of lesion,surgery,rectal temperature,pulse,total protein,outcome


In [76]:
horse8[horse8['Age'].isna()==True]
#нет пропусков

,Hospital Number,Age,type of lesion,surgery,rectal temperature,pulse,total protein,outcome


In [117]:
horse8[horse8['surgery'].isna()==True].info()
#один пропуск, можно просто удалить
#начнем формирование датафрема horse_without_NaN без пропусков
horse_without_NaN = horse8.drop(index = [131])
horse_without_NaN[horse_without_NaN['surgery'].isna()==True].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1 entries, 131 to 131
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Hospital Number     1 non-null      int64  
 1   Age                 1 non-null      int64  
 2   type of lesion      1 non-null      int64  
 3   surgery             0 non-null      float64
 4   rectal temperature  1 non-null      float64
 5   pulse               1 non-null      float64
 6   total protein       1 non-null      float64
 7   outcome             0 non-null      float64
dtypes: float64(5), int64(3)
memory usage: 72.0 bytes
<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Hospital Number     0 non-null      int64  
 1   Age                 0 non-null      int64  
 2   type of lesion      0 non-null      int64  
 3   surgery   

In [120]:
horse8[horse8['rectal temperature'].isna()==True].info()
#Заменим медианой
horse_without_NaN['rectal temperature'].fillna(horse_without_NaN['rectal temperature'].median(), inplace = True)
horse_without_NaN[horse_without_NaN['rectal temperature'].isna()==True].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60 entries, 4 to 294
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Hospital Number     60 non-null     int64  
 1   Age                 60 non-null     int64  
 2   type of lesion      60 non-null     int64  
 3   surgery             60 non-null     float64
 4   rectal temperature  0 non-null      float64
 5   pulse               42 non-null     float64
 6   total protein       48 non-null     float64
 7   outcome             60 non-null     float64
dtypes: float64(5), int64(3)
memory usage: 4.2 KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Hospital Number     0 non-null      int64  
 1   Age                 0 non-null      int64  
 2   type of lesion      0 non-null      int64  
 3   surgery        

In [121]:
horse8[horse8['pulse'].isna()==True].info()
#Заменим медианой
horse_without_NaN['pulse'].fillna(horse_without_NaN['pulse'].median(), inplace = True)
horse_without_NaN[horse_without_NaN['pulse'].isna()==True].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24 entries, 4 to 287
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Hospital Number     24 non-null     int64  
 1   Age                 24 non-null     int64  
 2   type of lesion      24 non-null     int64  
 3   surgery             24 non-null     float64
 4   rectal temperature  6 non-null      float64
 5   pulse               0 non-null      float64
 6   total protein       17 non-null     float64
 7   outcome             24 non-null     float64
dtypes: float64(5), int64(3)
memory usage: 1.7 KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Hospital Number     0 non-null      int64  
 1   Age                 0 non-null      int64  
 2   type of lesion      0 non-null      int64  
 3   surgery        

In [122]:
horse8[horse8['total protein'].isna()==True].info()
#Заменим медианой
horse_without_NaN['total protein'].fillna(horse_without_NaN['total protein'].median(), inplace = True)
horse_without_NaN[horse_without_NaN['total protein'].isna()==True].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33 entries, 4 to 295
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Hospital Number     33 non-null     int64  
 1   Age                 33 non-null     int64  
 2   type of lesion      33 non-null     int64  
 3   surgery             33 non-null     float64
 4   rectal temperature  21 non-null     float64
 5   pulse               26 non-null     float64
 6   total protein       0 non-null      float64
 7   outcome             33 non-null     float64
dtypes: float64(5), int64(3)
memory usage: 2.3 KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Hospital Number     0 non-null      int64  
 1   Age                 0 non-null      int64  
 2   type of lesion      0 non-null      int64  
 3   surgery        

In [240]:
horse8[horse8['outcome'].isna()==True]
#Это значение мы уже удалили

,Hospital Number,Age,type of lesion,surgery,rectal temperature,pulse,total protein,outcome
131,534572,1,3111,NaN,38.0,48.0,73.0,NaN


In [124]:
#Датафрейм без пропусков
horse_without_NaN.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 298 entries, 0 to 298
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Hospital Number     298 non-null    int64  
 1   Age                 298 non-null    int64  
 2   type of lesion      298 non-null    int64  
 3   surgery             298 non-null    float64
 4   rectal temperature  298 non-null    float64
 5   pulse               298 non-null    float64
 6   total protein       298 non-null    float64
 7   outcome             298 non-null    float64
dtypes: float64(5), int64(3)
memory usage: 21.0 KB
